# FSharp Samples

## Downloading Nuget Packages

In [1]:
#!fsharp
#r "nuget: XPlot.Plotly"
#r "nuget: FSharp.Data"
#r "nuget: MathNet.Numerics"
#r "nuget: Newtonsoft.Json"

Installed package Newtonsoft.Json version 12.0.3

Installed package MathNet.Numerics version 4.12.0

Installed package XPlot.Plotly version 3.0.1

Installed package FSharp.Data version 3.3.3

## Reading a CSV File

In [1]:
#!fsharp
open FSharp.Data
open System.IO

let sp500 = CsvFile.Load(Path.Join(__SOURCE_DIRECTORY__, "./Data/SP500.csv" ))

// Print the prices in the HLOC format
sp500.Rows
|> Seq.take 5

index,Columns
0,"[ 2008-05-02, 1413.900024, 0.0032303750170896706 ]"
1,"[ 2008-05-05, 1407.48999, -0.0045438914840980615 ]"
2,"[ 2008-05-06, 1418.26001, 0.007622806087752565 ]"
3,"[ 2008-05-07, 1392.569946, -0.018279853115200417 ]"
4,"[ 2008-05-08, 1397.680054, 0.0036628357410659618 ]"


## Plots

### Plotting a Pie Chart

In [1]:
#!fsharp
open XPlot.Plotly

[ "Pizza remaining", 3; "Pizza eaten", 5 ]
|> Chart.Pie

### Plotting a Line Plot

In [1]:
#!fsharp
open XPlot.Plotly
// Source: https://gist.github.com/theprash/7b6163d55b199359cd0dc68c477af0fa 

#r "System.Net.Http"
#r "nuget: Newtonsoft.Json"

let client = new System.Net.Http.HttpClient()
let weatherJson =
    client
        .GetStringAsync("http://www.7timer.info/bin/astro.php?lon=113.2&lat=23.1&ac=0&unit=metric&output=json&tzshift=0")
        .Result

let jObj = Newtonsoft.Json.Linq.JObject.Parse weatherJson

jObj.["dataseries"]
|> Seq.map (fun x -> x.Value<float> "timepoint", x.Value<float> "cloudcover" )
|> Seq.toArray
|> Chart.Line
|> Chart.WithXTitle "Time"
|> Chart.WithYTitle "Cloud cover"

### Plotting 2 Line Plots on One Chart

In [1]:
#!fsharp
open XPlot.Plotly
// Source: https://fslab.org/XPlot/chart/plotly-line-scatter-plots.html

let trace1 =
    Scatter(
        x = [1; 2; 3; 4],
        y = [10; 15; 13; 17]
    )

let trace2 =
    Scatter(
        x = [2; 3; 4; 5],
        y = [16; 5; 11; 9]
    )

[trace1; trace2]
|> Chart.Plot
|> Chart.WithWidth 700
|> Chart.WithHeight 500

### Plotting a Histogram

In [1]:
#!fsharp
open XPlot.Plotly
open MathNet.Numerics.Distributions

let x = 
    ContinuousUniform().Samples()
    |> Seq.take 10000
    |> Seq.toArray

Histogram(x = x) 
|> Chart.Plot
|> Chart.WithWidth 700
|> Chart.WithHeight 500

## Metropolis-Hastings


### Iterative Random Walk Metropolis Hastings

In [1]:
#!fsharp
open System
open MathNet.Numerics.Distributions
open XPlot.Plotly

let gaussianMetropolisHastingsFixedProposal (numberOfIterations: int) 
                                            (mu : double) 
                                            (sigma : double) : double list =
    let mutable states : double list = [] 
    let burnin : int = int (Math.Ceiling(float (double numberOfIterations * 0.2)))

    let proposedUniform = ContinuousUniform(mu - 5.0 * sigma, mu + 5.0 * sigma)
    let zeroOneUniform  = ContinuousUniform()
    let normal          = Normal(mu, sigma)

    // Initialize the current state.
    let mutable current = proposedUniform.Sample()

    for i in 0..numberOfIterations do

        // Proposed Move
        let proposedMove = proposedUniform.Sample()

        // Acceptance Ratio
        let currentProbability = normal.Density(current)
        let proposedProbabilty = normal.Density(proposedMove) 
        let acceptanceRatio    = Math.Min(proposedProbabilty / currentProbability, 1.0)

        // Choice
        let uniformDraw = zeroOneUniform.Sample() 

        if (uniformDraw < acceptanceRatio) then current <- proposedMove

        // Append the current state 
        states <- states @ [current]

    states
    |> List.skip(burnin)

let mhRun = gaussianMetropolisHastingsFixedProposal 50000 0.0 1.0

Histogram(x = mhRun)
|> Chart.Plot
|> Chart.WithWidth 700
|> Chart.WithHeight 500

## Parsing and Parameterizing the Model with a Sampling Model 

### Parsing the Model Pattern Matching

In [1]:
#!fsharp
open System

type ParsedRandomVariable = 
    { Name             : string; 
      Conditionals     : string list; 
      Distribution     : string; 
      Parameters       : string list; 
      Observed         : string option }
type ParsedBayesianModel = ParsedRandomVariable list

// Format: RVName [|Conditionals] ~ Distribution( Parameters ) [: observed] 
// [] -> optional
let parseLineOfModel (lineInModel : string) : ParsedRandomVariable = 
 
    // Helper fn to split the string based on a variety of type of delimiters.
    // Resultant type is a list of strings to feed in for the pattern matching.
    let splitToList (toSplit : string) (delimiters : obj) : string list = 
        let split = 
            match delimiters with
            | :? string        as s   -> toSplit.Split(s, StringSplitOptions.RemoveEmptyEntries) 
            | :? array<string> as arr -> toSplit.Split(arr, StringSplitOptions.RemoveEmptyEntries) 
            | :? array<char>   as arr -> toSplit.Split(arr, StringSplitOptions.RemoveEmptyEntries) 
            | _ -> failwithf "Splitting based on delimiters failed as it is neither a string nor an array of strings: Of Type: %A - %A" (delimiters.GetType()) toSplit
        
        Array.toList split

    match splitToList lineInModel " " with
    | nameAndConditionals :: "~" :: distributionParametersObserved ->
        // Get the name and conditionals.
        let splitNameAndConditionals = splitToList nameAndConditionals "|"
        let name = splitNameAndConditionals.[0]
        let conditionals = 
            match splitNameAndConditionals with 
            | name :: conditionals -> 
                if conditionals.Length > 0 then splitToList conditionals.[0] ","
                else []
            | _ -> failwithf "Pattern not found for RV Name and Conditionals - the format is: RVName|Condtionals: %A" splitNameAndConditionals

        let extractAndGetParameters (distributionNameAndParameters : string) : string * string list = 
            let splitDistributionAndParameters = splitToList distributionNameAndParameters [| "("; ")" |]
            (splitDistributionAndParameters.[0], splitToList splitDistributionAndParameters.[1] ",")
            
        match distributionParametersObserved with 

        // Case: Without Observations. Example: theta ~ Gamma(a,b)
        | distributionNameAndParameters when distributionNameAndParameters.Length = 1 ->
            let extractedDistributionAndParameters = extractAndGetParameters distributionNameAndParameters.[0]
            { Name             = name; 
              Conditionals     = conditionals; 
              Distribution     = (fst extractedDistributionAndParameters).ToLower();
              Observed         = None; 
              Parameters       = snd extractedDistributionAndParameters; }

        // Case: With Observations. Example: Y|theta ~ Poisson(theta) : observed
        | distributionNameAndParameters :: ":" :: observed ->
            let extractedDistributionAndParameters = extractAndGetParameters distributionNameAndParameters
            { Name             = name;
              Conditionals     = conditionals; 
              Distribution     = (fst extractedDistributionAndParameters).ToLower();
              Observed         = Some observed.Head; 
              Parameters       = snd extractedDistributionAndParameters; } 

        // Case: Error.
        | _ -> failwithf "Pattern not found for the model while parsing the distribution, parameters and optionally, the observed variables: %A" distributionParametersObserved 

    | _ -> failwithf "Pattern not found for the following line in the model - please check the syntax: %A" lineInModel

let parseModel (model : string) : ParsedBayesianModel = 
    model.Split('\n') 
    |> Array.toList
    |> List.map(parseLineOfModel)

let printParsedModel (model : string) : unit = 
    let parsedModel = parseModel model
    printfn "Model: %A is represented as %A" model parsedModel

In [1]:
#!fsharp
// Print out our simple 1-Parameter Model.
let model1 = @"theta ~ Gamma(a,b)
              Y|theta ~ Poisson(theta) : observed"
printParsedModel(model1)

let model2 = @"theta ~ Beta(unit,unit)
               Y|theta ~ Binomial(n,theta) : observed"
printParsedModel(model2)

// This model doesn't make sense but adding to test multiple conditionals.
let model3 = @"theta ~ Beta(unit,unit)
               gamma ~ Gamma(a,b)
               Y|theta,gammma ~ Binomial(n,theta) : observed"
printParsedModel(model3)

Model: 

"theta ~ Gamma(a,b)
              Y|theta ~ Poisson(theta) : observed"

 is represented as 

[{ Name = "theta"
   Conditionals = []
   Distribution = "gamma"
   Parameters = ["a"; "b"]
   Observed = None }; { Name = "Y"
                        Conditionals = ["theta"]
                        Distribution = "poisson"
                        Parameters = ["theta"]
                        Observed = Some "observed" }]

Model: 

...

[{ Name = "theta"
   Conditionals = []
   Distribution = "beta"
   Parameters = ["unit"; "unit"]
   Observed = None }; { Name = "Y"
                        Conditionals = ["theta"]
                        Distribution = "binomial"
                        Parameters = ["n"; "theta"]
                        Observed = Some "observed" }]

Model: 

"theta ~ Beta(unit,unit)
               gamma ~ Gamma(a,b)
               Y|theta,gammma ~ Binomial(n,theta) : observed"

 is represented as 

[{ Name = "theta"
   Conditionals = []
   Distribution = "beta"
   Parameters = ["unit"; "unit"]
   Observed = None }; { Name = "gamma"
                        Conditionals = []
                        Distribution = "gamma"
                        Parameters = ["a"; "b"]
                        Observed = None }; { Name = "Y"
                                             Conditionals = ["theta"; "gammma"]
                                             Distribution = "binomial"
                                             Parameters = ["n"; "theta"]
                                             Observed = Some "observed" }]

### Parameterization

In [1]:
#!fsharp
#r "nuget: Newtonsoft.Json" 
#r "nuget: MathNet.Numerics"

open System
open System.Collections.Generic

open MathNet.Numerics.Distributions

open Newtonsoft.Json

type Observed = float list

type ParameterList = 
    { Observed : float list; Parameters : Dictionary<string, float> } 

let deserializeParameters (paramsAsString : string) : ParameterList = 
    JsonConvert.DeserializeObject<ParameterList>(paramsAsString)

// Parsing parameters
let parameters1 = "{Parameters : {mu0 : 0, sigma0 : 1, mu : 5, sigma : 2, lambda : 4}, observed : [4,5,2]}"
let deserializedParameters1 = deserializeParameters parameters1
printfn "%A" ( deserializedParameters1 )

let parameters2 = "{Parameters: {lambda : 2}}"
let deserializedParameters2 = deserializeParameters parameters2

// Applying the Parameters to a Distribution
let exp = Exponential deserializedParameters2.Parameters.["lambda"] 
exp.Sample()

{ Observed = [4.0; 5.0; 2.0]
  Parameters = seq [[mu0, 0]; [sigma0, 1]; [mu, 5]; [sigma, 2]; ...] }

0.15175637365143071

### Domain of Distributions

In [1]:
#!fsharp
open MathNet.Numerics.Distributions

type DistributionType = 
    | Continuous
    | Discrete

type Parameter     = float
type DiscreteInput = int
type Input         = float

type DensityType = 
    | OneParameter         of (Parameter * Input -> float) 
    | OneParameterDiscrete of (Parameter * DiscreteInput -> float)
    | TwoParameter         of (Parameter * Parameter * Input -> float)

type DistributionInfo = { RVName             : string; 
                          DistributionType   : DistributionType 
                          DistributionName   : string; 
                          Parameters         : float list; 
                          Density            : DensityType } with
    member this.ComputeOneParameterPDF (parameter : float) (input : float) : float =
        match this.Density with
        | OneParameter pdf -> pdf(parameter,input)
        | _ -> failwithf "Incorrect usage of function with a non One Parameter Density Type. Density given: %A" this.Density 
    member this.ComputeOneParameterDiscretePMF (parameter : float) (input : int) : float =
        match this.Density with
        | OneParameterDiscrete pdf -> pdf(parameter,input)
        | _ -> failwithf "Incorrect usage of function with a non One Parameter Discrete Density Type. Density given: %A" this.Density 
    member this.ComputeTwoParameterPDF (parameter1 : float) (parameter2 : float ) (input : float) : float =
        match this.Density with
        | TwoParameter pdf -> pdf(parameter1, parameter2, input)
        | _ -> failwithf "Incorrect usage of function with a non Two Parameter Density Type. Density given: %A" this.Density 

let getDistributionInfoForRV (item : ParsedRandomVariable) 
                             (parameterList : ParameterList) : DistributionInfo = 
    // I know this is ugly but this functionality assumes the user enters the 
    // parameters in the order that's expected by the MathNet Numerics Library. 
    // Grab the parameters associated with this Random Variable.
    let rvParameters = 
        item.Parameters
        |> List.filter(parameterList.Parameters.ContainsKey) 
        |> List.map(fun item -> parameterList.Parameters.[item])

    // Extract Distribution Associated with the Parsed Random Variable.
    match item.Distribution with

    // 1 Parameter Distributions 
    | "exponential" -> 
        { RVName           = item.Name;
          DistributionName = item.Distribution; 
          Parameters       = rvParameters; 
          DistributionType = DistributionType.Continuous;
          Density          = OneParameter Exponential.PDF; }
    | "poisson" ->
        { RVName           = item.Name;
          DistributionName = item.Distribution; 
          Parameters       = rvParameters;
          DistributionType = DistributionType.Discrete;
          Density          = OneParameterDiscrete Poisson.PMF } 
    // 2 Parameter Distributions 
    | "normal" ->
        { RVName           = item.Name;
          DistributionName = item.Distribution; 
          Parameters       = rvParameters; 
          DistributionType = DistributionType.Continuous;
          Density          = TwoParameter Normal.PDF; }
    | "gamma" ->
        { RVName           = item.Name;
          DistributionName = item.Distribution; 
          Parameters       = rvParameters; 
          DistributionType = DistributionType.Continuous;
          Density          = TwoParameter Gamma.PDF; }
    | "beta" ->
        { RVName           = item.Name;
          DistributionName = item.Distribution; 
          Parameters       = rvParameters; 
          DistributionType = DistributionType.Continuous;
          Density          = TwoParameter Beta.PDF; }
    | "continuousuniform" ->
        { RVName           = item.Name;
          DistributionName = item.Distribution; 
          Parameters       = rvParameters; 
          DistributionType = DistributionType.Continuous;
          Density          = TwoParameter ContinuousUniform.PDF; }
    // Failure Case
    | _ -> failwithf "Distribution not registered: %A" item.Distribution

let getDistributionInfoForModel(model : string) (parameterList : string) : DistributionInfo list = 
    let parsedModel   = parseModel model
    let parameterList = deserializeParameters parameterList 
    parsedModel
    |> List.map(fun x -> getDistributionInfoForRV x parameterList)

let getDensityOrProbabilityForModel (model : string) 
                                    (parameterList : string) 
                                    (data : float seq) : IDictionary<string, float seq> = 
    getDistributionInfoForModel model parameterList
    |> List.map(fun (e : DistributionInfo) -> 
        match e.Density with
        | OneParameter p ->
            let param   = List.exactlyOne e.Parameters 
            let results = data |> Seq.map(fun d -> e.ComputeOneParameterPDF param d)
            e.RVName, results
        | OneParameterDiscrete p ->
            let param   = List.exactlyOne e.Parameters 
            let results = data |> Seq.map(fun d -> e.ComputeOneParameterDiscretePMF param (int d))
            e.RVName, results
        | TwoParameter p ->
            let p2 : float list = e.Parameters |> List.take 2
            let results = data |> Seq.map(fun d -> e.ComputeTwoParameterPDF p2.[0] p2.[1] d)
            e.RVName, results)
    |> dict

In [1]:
#!fsharp
// Exponential. 
let exponentialModel     = "x ~ Exponential(lambda)"
let exponentialParamList = "{Parameters: {lambda : 2, a : 2., b : 2.3 }, Observed : []}"
let exponentialDummyData = ContinuousUniform.Samples(0., 200.) |> Seq.take 2000
let exponentialPdfs      = getDensityOrProbabilityForModel exponentialModel exponentialParamList exponentialDummyData
printfn "Exponential: %A" (exponentialPdfs.Values)

// Normal.
let normalModel     = "x ~ Normal(mu,sigma)"
let normalParamList = "{Parameters: {mu: 0., sigma : 1.}, Observed : []}"
let normalDummyData = Normal.Samples(0.0, 1.0) |> Seq.take 2000
let normalPdfs      = getDensityOrProbabilityForModel normalModel normalParamList normalDummyData 
printfn "Normal: %A" (normalPdfs.Values)

// Poisson.
let poissonModel     = "x ~ Poisson(theta)"
let poissonParamList = "{Parameters: {theta: 44}, Observed : []}"
let poissonDummyData = ContinuousUniform.Samples(0., 5.) |> Seq.take 2000 
let poissonPdfs      = getDensityOrProbabilityForModel poissonModel poissonParamList poissonDummyData 
printfn "Poisson: %A" (poissonPdfs.Values)

Exponential: 

seq
  [seq
     [1.431739226e-141; 8.181263096e-14; 7.577958998e-137; 2.363794918e-05; ...]]

Normal: 

seq [seq [0.04706671226; 0.09539697767; 0.3665050201; 0.3055761132; ...]]

Poisson: 

seq
  [seq [7.781132241e-20; 7.781132241e-20; 1.104713281e-15; 7.532136009e-17; ...]]

In [1]:
#!fsharp
open XPlot.Plotly

Histogram(x = normalDummyData)
|> Chart.Plot
|> Chart.WithWidth 700
|> Chart.WithHeight 500

## Constructing the Bayesian Network 

### Bayesian Node

In [1]:
#!fsharp
type BayesianNodeTypeInfo =
| Observed of float list 
| NonObserved

type BayesianNode = 
    { Name                 : string; 
      NodeType             : BayesianNodeTypeInfo 
      DistributionInfo     : DistributionInfo
      ParsedRandomVariable : ParsedRandomVariable } with

    static member ConstructNode(parsedRandomVariable : ParsedRandomVariable)
                               (parameterList : ParameterList) =

        let nodeType : BayesianNodeTypeInfo =
            match parsedRandomVariable.Observed with
            | Some _ -> BayesianNodeTypeInfo.Observed parameterList.Observed
            | None   -> BayesianNodeTypeInfo.NonObserved
        { Name                 = parsedRandomVariable.Name;
          NodeType             = nodeType;
          DistributionInfo     = getDistributionInfoForRV parsedRandomVariable parameterList;
          ParsedRandomVariable = parsedRandomVariable; }

    member this.GetDependents (parsedBayesianModel : ParsedBayesianModel) : ParsedRandomVariable list =
        parsedBayesianModel 
        |> List.filter(fun x -> x.Conditionals |> List.contains(this.Name))

In [1]:
#!fsharp
let lineOfModel        = @"x ~ Exponential(lambda) : observed"
let paramList          = "{Parameters: {lambda : 2, a : 2., b : 2.3 }, observed : [1,2,3,55]}"
BayesianNode.ConstructNode (parseLineOfModel lineOfModel) (deserializeParameters paramList)

Name,NodeType,DistributionInfo,ParsedRandomVariable
x,Observed [1.0; 2.0; 3.0; 55.0],"{ RVName = ""x"" DistributionType = Continuous DistributionName = ""exponential"" Parameters = [2.0] Density = OneParameter <fun:getDistributionInfoForRV@53> }","{ Name = ""x"" Conditionals = [] Distribution = ""exponential"" Parameters = [""lambda""] Observed = Some ""observed"" }"


### Bayesian Network Model

In [1]:
#!fsharp
(* Will be built on later... DAG used to get the posterior *)
type BayesianNetworkModel =
    { Name    : string;
      Nodes   : IDictionary<string, BayesianNode>
      Network : IDictionary<string, BayesianNode list> } with

    static member ConstructModel (name : string)
                                 (model : ParsedBayesianModel)
                                 (parameterList : ParameterList) = 

            // Construct all the modes of the model.
            let allNodes : IDictionary<string, BayesianNode> = 
                model
                |> List.map(fun m -> m.Name, BayesianNode.ConstructNode m parameterList)
                |> dict

            // From the list of all models, for each node get the dependends and append to the network.
            let network = Dictionary<string, BayesianNode list>()

            for KeyValue(name, bayesianNode) in allNodes do
              let dependents = bayesianNode.GetDependents model
              let dependentsAsBayesianNodes = 
                dependents
                |> List.map(fun d -> allNodes.[d.Name])
              network.Add(name, dependentsAsBayesianNodes)
            { Name    = name;
              Nodes   = allNodes;
              Network = network }

### Simple Bayesian Network Model

In [1]:
#!fsharp
// Only to be used for a model with 2 nodes 
// i.e. one for the prior and one for the likelihood.
type SimpleBayesianNetworkModel = 
  { Name       : string;
    Nodes      : IDictionary<string, BayesianNode>
    Prior      : BayesianNode;
    Likelihood : BayesianNode; } with

    member this.GetPriorProbability (input : float): float = 
      let distributionInfo = this.Prior.DistributionInfo 
      match distributionInfo.Density with
      | OneParameter p ->
          let param = List.exactlyOne distributionInfo.Parameters 
          distributionInfo.ComputeOneParameterPDF param input 
      | OneParameterDiscrete p ->
          let param   = List.exactlyOne distributionInfo.Parameters 
          distributionInfo.ComputeOneParameterDiscretePMF param (int input)
      | TwoParameter p ->
          let p2 : float list = distributionInfo.Parameters |> List.take 2
          distributionInfo.ComputeTwoParameterPDF p2.[0] p2.[1] input

    member this.GetLikelihoodProbability (prior : float) : float = 
        let distributionInfo = this.Likelihood.DistributionInfo
        let observed         = match this.Likelihood.NodeType with
                               | Observed l -> l
                               | _ -> failwithf "Incorrectly constructed Simple Network Model. %A" this 
        let density = 
          match distributionInfo.Density with
          | OneParameter p -> 
              observed |> List.map(fun d -> distributionInfo.ComputeOneParameterPDF prior d)
          | OneParameterDiscrete p ->
              observed |> List.map(fun d -> distributionInfo.ComputeOneParameterDiscretePMF prior (int (Math.Ceiling d)))
          | TwoParameter p ->
              let p : float = distributionInfo.Parameters |> List.exactlyOne 
              observed |> List.map(fun d -> distributionInfo.ComputeTwoParameterPDF prior p d)

        density
        |> List.fold (*) 1.0 

    member this.GetPosteriorWithoutScalingFactor (input: float) : float = 
      let priorPdf      = this.GetPriorProbability input 
      let likelihoodPdf = this.GetLikelihoodProbability priorPdf 
      priorPdf * likelihoodPdf

    static member ConstructModel (name : string)
                                 (model : ParsedBayesianModel)
                                 (parameterList : ParameterList) : SimpleBayesianNetworkModel = 

        // Construct all the modes of the model.
        let allNodes : (string * BayesianNode) list =
            model
            |> List.map(fun m -> m.Name, BayesianNode.ConstructNode m parameterList)

        let prior : BayesianNode =  
            allNodes
            |> List.filter(fun (_,m) -> m.NodeType = BayesianNodeTypeInfo.NonObserved) 
            |> List.map(fun (_,m) -> m)
            |> List.exactlyOne

        let likelihood : BayesianNode =  
            allNodes
            |> List.filter(fun (_,m) -> m.NodeType <> BayesianNodeTypeInfo.NonObserved)
            |> List.map(fun (_,m) -> m)
            |> List.exactlyOne

        { Name       = name; 
          Nodes      = dict allNodes; 
          Prior      = prior;
          Likelihood = likelihood; } 

In [1]:
#!fsharp
let model = @"x ~ Gamma(a,b)
              y|x ~ Poisson(x) : observed"
let parsedModel = parseModel model
let paramList = "{Parameters: {a : 2., b : 2.3}, observed : [15, 20, 22, 2,3,55]}"
SimpleBayesianNetworkModel.ConstructModel "Poisson Model" parsedModel (deserializeParameters paramList)

Name,Nodes,Prior,Likelihood
Poisson Model,"[ [x, { Name = ""x"" NodeType = NonObserved DistributionInfo = { RVName = ""x"" DistributionType = Continuous DistributionName = ""gamma"" Parameters = [2.0; 2.3] Density = TwoParameter <fun:getDistributionInfoForRV@72-3> } ParsedRandomVariable = { Name = ""x"" Conditionals = [] Distribution = ""gamma"" Parameters = [""a""; ""b""] Observed = None } }], [y, { Name = ""y"" NodeType = Observed [15.0; 20.0; 22.0; 2.0; 3.0; 55.0] DistributionInfo = { RVName = ""y"" DistributionType = Discrete DistributionName = ""poisson"" Parameters = [] Density = OneParameterDiscrete <fun:getDistributionInfoForRV@59-1> } ParsedRandomVariable = { Name = ""y"" Conditionals = [""x""] Distribution = ""poisson"" Parameters = [""x""] Observed = Some ""observed"" } }] ]","{ Name = ""x"" NodeType = NonObserved DistributionInfo = { RVName = ""x"" DistributionType = Continuous DistributionName = ""gamma"" Parameters = [2.0; 2.3] Density = TwoParameter <fun:getDistributionInfoForRV@72-3> } ParsedRandomVariable = { Name = ""x"" Conditionals = [] Distribution = ""gamma"" Parameters = [""a""; ""b""] Observed = None } }","{ Name = ""y"" NodeType = Observed [15.0; 20.0; 22.0; 2.0; 3.0; 55.0] DistributionInfo = { RVName = ""y"" DistributionType = Discrete DistributionName = ""poisson"" Parameters = [] Density = OneParameterDiscrete <fun:getDistributionInfoForRV@59-1> } ParsedRandomVariable = { Name = ""y"" Conditionals = [""x""] Distribution = ""poisson"" Parameters = [""x""] Observed = Some ""observed"" } }"


## Inference

### Computing the Unscaled Posterior with an Arbitary Input.

In [1]:
#!fsharp
let model = @"x ~ Gamma(a,b) 
              y|x ~ Poisson(x) : observed"
let parsedModel = parseModel model
let paramList = "{Parameters: { a : 2., b : 2 }, observed : [1,2,3,4]}"
let simpleModel = SimpleBayesianNetworkModel.ConstructModel "Poisson Model" parsedModel (deserializeParameters paramList)
simpleModel.GetPosteriorWithoutScalingFactor 2.

1.2915165523285174E-12

### What the Posterior Distribution Should Look Like

In [1]:
#!fsharp
open XPlot.Plotly

let conjugatePriorBasedPosterior = 
    let gamma = Gamma(shape = 12., rate = 6.) // Gamma( a + sum( Y_i ), b + n )
    let samples = gamma.Samples() |> Seq.take 10000

    Histogram(x = samples) 
    |> Chart.Plot
    |> Chart.WithWidth 700
    |> Chart.WithHeight 500

conjugatePriorBasedPosterior

### MCMC Domain 

In [1]:
#!fsharp
type ConvergenceCriteria = 
    | IterativeConvergence of int

type ProposalDistribution =
    | Normal of float  // Normal( current, delta )
    | ContinuousUniform of float // ContinuousUniform( current - delta, current + delta )
    | PositiveContinousUniform of float // ( x ~ Uniform( current - delta, current + delta ) if x <= 0 then 0.1 else x )

type MCMCInferenceStep = 
    | SymmetricMetropolisHastings of ProposalDistribution

type MCMCChain =
    { Id             : int
      AcceptanceRate : float
      StepValues     : seq<float> }

type MCMCRequest = 
    { StepFunction         : MCMCInferenceStep
      ConvergenceCriteria  : ConvergenceCriteria
      BurnInIterationsPct  : float 
      Chains               : int 
      PrintDebug           : bool }

type MCMCResult =
    { Chains         : seq<MCMCChain> 
      MCMCRequest    : MCMCRequest }

### Computing the Posterior 

In [1]:
#!fsharp
open System
open MathNet.Numerics.Distributions
open XPlot.Plotly

type AcceptanceRejection =
    | Acceptance of float
    | Rejection  of float 

let doSymmetricMetropolisHastings (request : MCMCRequest) 
                                  (iterations : int)
                                  (proposalDistribution : ProposalDistribution)
                                  (simpleBayesianModel : SimpleBayesianNetworkModel) : MCMCResult =

    let getChain (id : int) (request : MCMCRequest) 
                 (iterations : int) (simpleBayesianModel : SimpleBayesianNetworkModel) : MCMCChain =

        let burnin         = int (Math.Ceiling(request.BurnInIterationsPct / 100. * float iterations))
        let zeroOneUniform = ContinuousUniform()

        let mutable current = simpleBayesianModel.GetPosteriorWithoutScalingFactor 1. // TODO: Figure out a good starting value.. 

        let matchedProposalDistribution (input : float) :  float =
            match proposalDistribution with 
            | Normal delta            -> Normal(input, delta).Sample()
            | ContinuousUniform delta -> ContinuousUniform(input - delta, input + delta).Sample()
            | PositiveContinousUniform delta -> 
                let u = ContinuousUniform(input - delta, input + delta).Sample()
                if u <= 0. then input + 0.1 else u

        let step (iteration : int) : AcceptanceRejection =
            let proposed           = matchedProposalDistribution current
            let currentProbability = simpleBayesianModel.GetPosteriorWithoutScalingFactor current
            let proposedProbabilty = simpleBayesianModel.GetPosteriorWithoutScalingFactor proposed
            let acceptanceRatio    = Math.Min(currentProbability / proposedProbabilty, 1.)
            let uniformDraw        = zeroOneUniform.Sample()

            if request.PrintDebug then 
                printfn "Chain: %A Iteration: %A - Current: %A | Proposed: %A | AcceptanceRatio: %A | Uniform Draw: %A | Current: %A" 
                    id iteration currentProbability proposedProbabilty acceptanceRatio uniformDraw current

            if uniformDraw < acceptanceRatio then (current <- proposed; Acceptance proposed) 
            else Rejection current
    
        let stepResults : AcceptanceRejection seq =
            seq {1..iterations}
            |> Seq.map step
            |> Seq.skip burnin

        let getAcceptanceRateAndStepValues : float * float seq =

            // Compute the Acceptance Rate.
            let acceptanceRate : float = 
                let totalBurninWithoutBurnin : float = float (Seq.length stepResults)
                let totalNumberOfAcceptances : float = 
                    stepResults 
                    |> Seq.filter(fun x -> 
                        match x with
                        | Acceptance x -> true
                        | _ -> false)
                    |> Seq.length
                    |> float
                totalNumberOfAcceptances / totalBurninWithoutBurnin

            // Grab the Step Values that'll approximate the posterior.
            let stepValues =
                stepResults |> Seq.map(fun s ->
                    match s with
                    | Acceptance v -> v
                    | Rejection v -> v)

            acceptanceRate, stepValues

        let acceptanceRate, stepValues = getAcceptanceRateAndStepValues 

        { Id             = id
          AcceptanceRate = acceptanceRate
          StepValues     = stepValues} 

    let chains : MCMCChain seq =
        seq {1..request.Chains}
        |> Seq.map(fun id ->  getChain id request iterations simpleBayesianModel)

    { Chains      = chains
      MCMCRequest =  request }

let runMCMC (request : MCMCRequest) 
            (model : SimpleBayesianNetworkModel) : MCMCResult =
    match request.StepFunction with
    | SymmetricMetropolisHastings proposalDistribution ->
        match request.ConvergenceCriteria with 
        | IterativeConvergence iterations -> 
            doSymmetricMetropolisHastings request iterations proposalDistribution model
        | _ -> failwith "You need to pass in the number of iterations for the Metropolis-Hastings algorithm"
    | _ -> failwithf "Step Function Not Registered: %A" request.StepFunction

In [1]:
#!fsharp
let model = @"x ~ Exponential(a) 
              y|x ~ Exponential(x) : observed"
let parsedModel = parseModel model
let paramList = "{Parameters: {a : 2., b : 2.}, observed : [1,2,3,4,4,2,5,6,7,3,2,3,4,5,6,1,2,3,4,4,4,4]}"
let simpleModel = SimpleBayesianNetworkModel.ConstructModel "Poisson Model" parsedModel (deserializeParameters paramList)

let request : MCMCRequest = 
    { StepFunction        = SymmetricMetropolisHastings (ProposalDistribution.PositiveContinousUniform 0.002)
      ConvergenceCriteria = IterativeConvergence 10000
      BurnInIterationsPct = 0.2
      PrintDebug          = false 
      Chains              = 4 }

let mcmc       = runMCMC request simpleModel
let firstChain = Seq.head mcmc.Chains

Histogram(x = firstChain.StepValues)
|> Chart.Plot
